In [1]:
import requests

In [2]:
import json

In [3]:
from bs4 import BeautifulSoup

In [4]:
resp = requests.get('http://moabb.neurotechx.com/docs/datasets.html')

In [5]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [6]:
webpage = soup.find_all('a')

In [7]:
all_datasets_links = webpage[42:74]

In [8]:
all_datasets_links.pop(16) # popping unrelated links

In [9]:
all_datasets_links.pop(23) # popping unrelated links

In [10]:
parts_of_links = [tag['href'] for tag in all_datasets_links]

In [11]:
links = ['http://moabb.neurotechx.com/docs/'+link for link in parts_of_links]

In [12]:
moabb_datasets = list()

In [13]:
for link in links:
    dataset = dict()
    print('requesting page with URL: '+link)
    dataset_page = requests.get(link)
    dataset_soup = BeautifulSoup(dataset_page.text, 'html.parser')
    dataset['Name of dataset'] = dataset_soup.title.string
    text = dataset_soup.get_text().replace("\n", "")
    if 'Dataset description' in text:
        dataset['Dataset description'] = text[text.index('Dataset description') + len('Dataset description'): text.index('References')]
    else:
        dataset['Dataset description'] = None
    if 'References' in text:
        dataset['References'] = text[text.index('References') + len('References'): text.index('Methods')]
    else:
        dataset['References'] = None
    if 'Methods' in text:
        if 'Examples' in text:
            dataset['Methods'] = text[text.index('Methods') + len('Methods'): text.index('Examples')]
        else:
            dataset['Methods'] = text[text.index('Methods') + len('Methods'): text.index('Back to top')]       
    else:
        dataset['Methods'] = None
    if 'Examples' in text:
        dataset['Examples'] = text[text.index('Examples'): text.index('Back to top')]
    else:
        dataset['Examples'] = None
    dataset['url'] = link
    print('adding data')
    moabb_datasets.append(dataset)
    

In [14]:
for dictionary in moabb_datasets:
    with open("moabb_datasets.json", "w") as outfile:
        json.dump(dictionary, outfile)
        

In [15]:
%notebook kerven_moabb_datasets.ipynb

In [16]:
%pip install nbformat